# OLS - Wooldridge Computer Exercise
## Chapter 7, Exercise 2

## To add a heading:
- Insert a new cell
- Type or paste-in content
- Place a single / just one "pound-sign" in front of the heading content
- Select "Markdown"
- Press "Shift", "Enter" at same time to convert to clean commentary

## To add a sub-heading:
- Insert a new cell
- Type or paste-in content
- Place two "pound-signs" in front of the sub-heading
- Select "Markdown"
- Press "Shift", "Enter" at same time to convert to clean commentary

## To add new bulleted documentation:

- Insert a new cell
- Type or paste-in content
- Place a "dash" character in front of the bulleted content
- Select "Markdown"
- Press "Shift", "Enter" at same time to convert to clean commentary

# References
- Wooldridge, J.M. (2016). Introductory econometrics: A modern approach (6thed.). Mason, OH: South-Western, Cengage Learning.
- Residual Plots: https://medium.com/@emredjan/emulating-r-regression-plots-in-python-43741952c034
- Understanding residual plots: https://data.library.virginia.edu/diagnostic-plots/
- VIF: https://etav.github.io/python/vif_factor_python.html
- VIF: https://en.wikipedia.org/wiki/Variance_inflation_factor
- Extracting various values from regression results: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html

# Instantiate libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms

from statsmodels.formula.api import ols
from statsmodels.compat import lzip
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.graphics.gofplots import ProbPlot

#import pandas.tseries.api as sm
#from tseries.formula.apt import ols

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr


plt.style.use('seaborn') # pretty matplotlib plots

plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=18)

# Latex markup language 
from IPython.display import Latex

# Data Read from csv

In [2]:
%%time
#df = pd.read_csv(BytesIO(csv_as_bytes),sep='|',nrows=100000)
df1 = pd.read_stata('C://Users//Family//Documents//DataSetEconomics//Wooldridge//WAGE2.dta')
print(df1.head())

   wage  hours   IQ  KWW  educ  exper  tenure  age  married  black  south  \
0   769     40   93   35    12     11       2   31        1      0      0   
1   808     50  119   41    18     11      16   37        1      0      0   
2   825     40  108   46    14     11       9   33        1      0      0   
3   650     40   96   32    12     13       7   32        1      0      0   
4   562     40   74   27    11     14       5   34        1      0      0   

   urban  sibs  brthord  meduc  feduc     lwage  
0      1     1      2.0    8.0    8.0  6.645091  
1      1     1      NaN   14.0   14.0  6.694562  
2      1     1      2.0   14.0   14.0  6.715384  
3      1     4      3.0   12.0   12.0  6.476973  
4      1    10      6.0    6.0   11.0  6.331502  
Wall time: 94.9 ms


In [3]:
df1['constant'] = 1

# Data Checks
- Columns

In [4]:
%%time
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935 entries, 0 to 934
Data columns (total 18 columns):
wage        935 non-null int16
hours       935 non-null int8
IQ          935 non-null int16
KWW         935 non-null int8
educ        935 non-null int8
exper       935 non-null int8
tenure      935 non-null int8
age         935 non-null int8
married     935 non-null int8
black       935 non-null int8
south       935 non-null int8
urban       935 non-null int8
sibs        935 non-null int8
brthord     852 non-null float64
meduc       857 non-null float64
feduc       741 non-null float64
lwage       935 non-null float32
constant    935 non-null int64
dtypes: float32(1), float64(3), int16(2), int64(1), int8(11)
memory usage: 53.9 KB
Wall time: 11 ms


# OLS: Regress lwage on educ, exper, exper^2

In [4]:
%%latex
\begin{align}
{Unrestricted Model: lwage = \alpha + \beta_{1} educ + \beta_{2} exper + \beta_{3} tenure + \beta_{4} married + \beta_{5} black + \beta_{6} south + \beta_{7} urban + \beta_{8} exper^{2} + \beta_{9} tenure^{2} + \mu}
\end{align}
\begin{align}
{Restricted Model:   lwage = \alpha + \beta_{1} educ + \beta_{2} exper + \beta_{3} tenure + \beta_{4} married + \beta_{5} black + \beta_{6} south + \beta_{7} urban + \mu}
\end{align}

<IPython.core.display.Latex object>

In [5]:
formula1 = '''lwage ~ educ + exper + tenure + married + black + south + urban'''
formula2 = '''lwage ~ educ + exper + tenure + married + black + south + urban + exper^2 + tenure^2'''
#model = ols(formula, df).fit(cov_type='HC0')
model1 = ols(formula1, df1)
results1 = model1.fit()
model2 = ols(formula2, df1)
results2 = model2.fit()
aov_table1 = statsmodels.stats.anova.anova_lm(results1, typ=2)
aov_table2 = statsmodels.stats.anova.anova_lm(results2, typ=2)
print(aov_table1)
print(aov_table2)
print(results1.summary())
print(results2.summary())
ssr_r = results1.ssr
ssr_ur = results2.ssr
df_ur = results2.df_resid
f_stat = ((ssr_r-ssr_ur) / 2) / (ssr_ur / results2.df_resid)
print()
print('F-Statistic: Restricted vs Unrestricted Models')
print(f_stat)

              sum_sq     df           F        PR(>F)
educ       14.637079    1.0  109.584353  2.576585e-24
exper       2.596312    1.0   19.437975  1.160827e-05
tenure      3.063340    1.0   22.934504  1.950493e-06
married     3.483253    1.0   26.078293  3.979232e-07
black       3.339820    1.0   25.004443  6.839311e-07
south       1.601988    1.0   11.993706  5.582258e-04
urban       6.216421    1.0   46.540871  1.617983e-11
Residual  123.818521  927.0         NaN           NaN
                sum_sq     df           F        PR(>F)
educ         14.483874    1.0  108.375037  4.474531e-24
exper         0.670685    1.0    5.018374  2.531648e-02
tenure        0.072486    1.0    0.542373  4.616378e-01
married       3.557180    1.0   26.616461  3.036167e-07
black         3.411232    1.0   25.524412  5.262804e-07
south         1.618733    1.0   12.112105  5.243551e-04
urban         6.114072    1.0   45.748316  2.381442e-11
exper ^ 2     0.004021    1.0    0.030090  8.623251e-01
tenure ^ 2

In [6]:
%%latex
\begin{align}
{Unrestricted Model: lwage = \alpha + \beta_{1} educ + \beta_{2} exper + \beta_{3} tenure + \beta_{4} married + \beta_{5} black + \beta_{6} south + \beta_{7} urban + \beta_{8} exper^{2} + \beta_{9} tenure^{2} + \mu}
\end{align}
\begin{align}
{Restricted Model:   lwage = \alpha + \beta_{1} educ + \beta_{2} exper + \beta_{3} tenure + \beta_{4} married + \beta_{5} black + \beta_{6} south + \beta_{7} urban + \mu}
\end{align}

<IPython.core.display.Latex object>

In [7]:
formula_r = '''lwage ~ educ + exper + tenure + married + black + south + urban'''
formula_ur = '''lwage ~ educ + exper + tenure + married + black + south + urban + exper^2 + tenure^2'''
#model = ols(formula, df).fit(cov_type='HC0')
model_ur = ols(formula_ur, df1)
results_ur = model_ur.fit()
model_r = ols(formula_r, df1)
results_r = model_r.fit()
print(results_ur.summary())
print()
print('F-Statistic: Restricted vs Unrestricted Models')
print(results_ur.compare_f_test(results_r))

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     34.95
Date:                Mon, 18 Mar 2019   Prob (F-statistic):           2.55e-53
Time:                        20:09:44   Log-Likelihood:                -380.81
No. Observations:                 935   AIC:                             781.6
Df Residuals:                     925   BIC:                             830.0
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3968      0.113     47.644      0.0

In [20]:
%%latex
\begin{align}
{lwage = \alpha + \beta_{1} educ + \beta_{2} exper + \beta_{3} tenure + \beta_{4} married + \beta_{5} black + \beta_{6} south + \beta_{7} urban + \beta_{8} educ \times black + \mu}
\end{align}

<IPython.core.display.Latex object>

In [12]:
formula = '''lwage ~ educ + exper + tenure + married + black + south + urban + educ:black'''
#model = ols(formula, df).fit(cov_type='HC0')
model = ols(formula, df1)
results = model.fit()
aov_table = statsmodels.stats.anova.anova_lm(results, typ=2)
print(aov_table)
print(results.summary())

                sum_sq     df           F        PR(>F)
educ         14.637079    1.0  109.614677  2.549391e-24
exper         2.507370    1.0   18.777283  1.629706e-05
tenure        3.083458    1.0   23.091511  1.801395e-06
married       3.465013    1.0   25.948908  4.248118e-07
black         3.339820    1.0   25.011362  6.816750e-07
south         1.547366    1.0   11.587970  6.922824e-04
urban         6.212356    1.0   46.523309  1.632855e-11
educ:black    0.167785    1.0    1.256514  2.626027e-01
Residual    123.650736  926.0         NaN           NaN
                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     39.32
Date:                Sun, 16 Dec 2018   Prob (F-statistic):           4.35e-54
Time:                        19:13:22   Log-L

In [18]:
df1.loc[df1['married'] == 1, 'single'] = 0
df1.loc[df1['married'] == 0, 'single'] = 1
df1.loc[df1['black'] == 1, 'nonblack'] = 0
df1.loc[df1['black'] == 0, 'nonblack'] = 1

formula = '''lwage ~ educ + exper + tenure + married:black + married:nonblack + single:nonblack + south + urban '''
#model = ols(formula, df).fit(cov_type='HC0')
model = ols(formula, df1)
results = model.fit()
aov_table = statsmodels.stats.anova.anova_lm(results, typ=2)
print(aov_table)
print(results.summary())

   wage  hours   IQ  KWW  educ  exper  tenure  age  married  black  south  \
0   769     40   93   35    12     11       2   31        1      0      0   
1   808     50  119   41    18     11      16   37        1      0      0   
2   825     40  108   46    14     11       9   33        1      0      0   
3   650     40   96   32    12     13       7   32        1      0      0   
4   562     40   74   27    11     14       5   34        1      0      0   

   urban  sibs  brthord  meduc  feduc     lwage  constant  single  nonblack  
0      1     1      2.0    8.0    8.0  6.645091         1     0.0       1.0  
1      1     1      NaN   14.0   14.0  6.694562         1     0.0       1.0  
2      1     1      2.0   14.0   14.0  6.715384         1     0.0       1.0  
3      1     4      3.0   12.0   12.0  6.476973         1     0.0       1.0  
4      1    10      6.0    6.0   11.0  6.331502         1     0.0       1.0  
                      sum_sq     df           F        PR(>F)
educ   